# Notebook 0001: Data Preparation

This simply takes a CSV version of a processed HMDA file, does some additional formatting and downcasting, and then saves it as a compressed parquet file. The parquet file will be used in subsequent notebooks. 

_Users do not need to run this notebook_.

In [1]:
import pandas as pd

pd.set_option('display.max_columns', None)

## Import CSV File

In [2]:
df = pd.read_csv('hmda.csv', index_col='Unnamed: 0')
df.index.name = 'ID'

## Downcasting Float and Integer Columns

This is done so that the exported parquet file is as small as possible.

In [3]:
float_cols = df.select_dtypes(include=[float]).columns.to_list()
print(f"Float Columns:\n{float_cols}")
int_cols = df.select_dtypes(include=[int]).columns.to_list()
print(f"\nInteger Columns:\n{int_cols}")
str_cols = df.select_dtypes(include=[object]).columns.to_list()
print(f"\nString Colummns:\n{str_cols}")

Float Columns:
['cv_fold', 'high_priced', 'low_priced', 'interest_rate', 'rate_spread', 'loan_amount', 'loan_to_value_ratio', 'property_value', 'income', 'debt_to_income_ratio', 'term_360', 'conforming', 'state_target_encoded', 'black', 'asian', 'white', 'native_american', 'hawaiian_or_pacific_islander', 'hispanic', 'non_hispanic', 'male', 'female', 'agegte62', 'agelt62']

Integer Columns:
['no_intro_rate_period', 'intro_rate_period', 'product_type_conventional', 'product_type_fha', 'product_type_fsa_rhs', 'product_type_va']

String Colummns:
['train', 'debt_to_income_ratio_original', 'state', 'product_type', 'race', 'ethnicity', 'sex']


In [4]:
for f_i in float_cols:
    df[f_i] = pd.to_numeric(df[f_i], downcast='float')
for i_i in int_cols:
    df[i_i] = pd.to_numeric(df[i_i], downcast='integer')
df = df.drop(columns=['cv_fold', 'high_priced', 'debt_to_income_ratio_original', 'state', 'product_type'])

## Format and Clean Group Information

In [5]:
group_vars = [
    'black', 'asian', 'white', 'native_american', 'hawaiian_or_pacific_islander', 
    'hispanic', 'non_hispanic', 'male', 'female', 'agegte62', 'agelt62'
]
renamer = dict(zip(group_vars, [x.replace('_', ' ').title() for x in group_vars]))
renamer['non_hispanic'] = 'Non-Hispanic'
renamer['agegte62'] = 'Age >= 62'
renamer['agelt62'] = 'Age < 62'
df = df.rename(columns=renamer)
df['Asian'] = df['Asian'] + df['Hawaiian Or Pacific Islander']
df = df.rename(columns={'Asian': 'Asian/PI'})
df = df.drop(columns=['Hawaiian Or Pacific Islander'])
df.loc[df['race'].isin(['Hawaiian or pacific islander', 'Asian']), 'race'] = 'Asian/PI'
df.loc[df['race'] == 'Native american', 'race'] = 'Native American'

## Save Parquet File For Subsequent Notebooks

In [6]:
df.to_parquet('hmda.parquet', compression='zstd', index=True)
del df

## Validate Export

In [7]:
data = pd.read_parquet('hmda.parquet')
display(data.sample(n=3, random_state=161803))

,train,low_priced,interest_rate,rate_spread,loan_amount,loan_to_value_ratio,no_intro_rate_period,intro_rate_period,property_value,income,debt_to_income_ratio,term_360,conforming,state_target_encoded,product_type_conventional,product_type_fha,product_type_fsa_rhs,product_type_va,Black,Asian/PI,White,Native American,Hispanic,Non-Hispanic,Male,Female,Age >= 62,Age < 62,race,ethnicity,sex
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1875923,valid,1.0,0.05375,0.00753,75000.0,1.0000,1,0,75000.0,36000.0,0.36,1.0,1.0,0.141784,1,0,0,0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,NaN,NaN,White,Non-Hispanic,Male
2389403,train,1.0,0.04500,0.00042,285000.0,0.6509,1,0,435000.0,75000.0,0.33,1.0,1.0,0.089620,1,0,0,0,0.0,0.0,1.0,0.0,0.0,1.0,NaN,NaN,1.0,0.0,White,Non-Hispanic,Unknown
2591428,train,1.0,0.05625,0.01348,55000.0,0.8000,1,0,65000.0,23000.0,0.33,1.0,1.0,0.125738,1,0,0,0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,White,Non-Hispanic,Male
